In [75]:
import os
import json
import numpy as np
import tensorflow as tf

In [76]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [77]:
# Name of directory for grids
directory_name = './data/big_grids/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_distance.append(i['distance'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()

In [78]:
# Name of directory for grids
directory_name = './data/big_grids_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_distance.append(i['distance'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [79]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 50, 50, 1) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 2, 1, 1) )
train_in_distance = np.reshape( x_train_distance, (-1,1,1, 1) ) / 60
train_in_locals = np.reshape( x_train_locals, (-1, 3, 3, 1) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 50, 50, 1) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 2, 1, 1) )
test_in_distance = np.reshape( x_test_distance, (-1, 1, 1, 1) ) / 60
test_in_locals = np.reshape( x_test_locals, (-1, 3, 3, 1) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [80]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (50,50,1) )
cnn_grid = tf.keras.layers.Conv2D( filters = 64, kernel_size = (3,3), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( grid_input )
flatten_grid = tf.keras.layers.Flatten()( cnn_grid )

In [81]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1,1) )
cnn_position = tf.keras.layers.Conv2D( filters = 5, kernel_size = (1,1), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( position_input )
flatten_position = tf.keras.layers.Flatten()( cnn_position )

In [82]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (3,3,1) )
cnn_local = tf.keras.layers.Conv2D( filters = 32, kernel_size = (3,3), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( local_input )
flatten_local = tf.keras.layers.Flatten()( cnn_local )

In [83]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_local, flatten_position])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 147498), dtype=tf.float32, name=None), name='concatenate_5/concat:0', description="created by layer 'concatenate_5'")


In [84]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 80, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 30, activation = tf.nn.relu )( dense_2 )
dense_4 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_3 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_4 )
probabilities = tf.keras.layers.Softmax()( logits )

In [85]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, local_input, position_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [86]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [87]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

0	1219	0	0
0	2953	5	0
0	7134	10	0
0	10511	7	0


In [88]:
# Train the model
history = model.fit( [train_in_grid, train_in_locals, train_in_position], train_out, epochs = 5 )

Epoch 1/5
4095/4095 [==============================] - 442s 108ms/step - loss: 0.8633 - accuracy: 0.6344
Epoch 2/5
4095/4095 [==============================] - 473s 116ms/step - loss: 0.3446 - accuracy: 0.8706
Epoch 3/5
4095/4095 [==============================] - 457s 112ms/step - loss: 0.2077 - accuracy: 0.9196
Epoch 4/5
4095/4095 [==============================] - 521s 127ms/step - loss: 0.1671 - accuracy: 0.9368
Epoch 5/5
4095/4095 [==============================] - 544s 133ms/step - loss: 0.1140 - accuracy: 0.9598


In [89]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

769	385	54	11
46	2808	103	1
53	24	7027	40
127	67	74	10250


In [90]:
model.save('./models/agent1_CNN')

2021-12-11 14:45:08.082145: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./models/agent1_CNN/assets
